### Trees Recommendations Proposal

#### 1. Introduction
This proposal will suggest a clear, mathematical method for scoring and ranking plans for an individual based on their current goals/requirements to reach their potential. The method will be based on the following assumptions:

- The individual has a set of goals/requirements that they want to achieve
- The individual has a set of plans that they can choose from to achieve their goals/requirements
- The individual has a set of attributes that can be used to evaluate them compared to other individuals
- The plan has a set of attributes that can be used to evaluate it compared to other plans

We will then implement a customizable method for combining these assumptions to generate a score for each plan for each individual. This score will be stored in the backend, in a user_plan_score array, which will be used to rank the plans for each individual, and a commonly hit endpoint will be created to return the top 5 plans for a given individual.

#### 2. Method
This section of the notebook will explore a potential method for scoring and ranking plans for each individual.

<hr>

In [1]:
!pip install psycopg2-binary sqlalchemy pandas

In [2]:
!pip install python-dotenv

In [3]:
!pip install SQLAlchemy==1.3.23

In [23]:
!pip install ipywidgets

     |████████████████████████████████| 139 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 214 kB 30.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.3 MB 52.4 MB/s eta 0:00:01


In [21]:
## Let's import the data and take a look at it
import psycopg2
import pandas as pd
import sqlalchemy
from dotenv import load_dotenv
import os

inspect = sqlalchemy.inspect

dotenv_path = 'envs/prod.env'  # replace with the path to your .env file
load_dotenv(dotenv_path=dotenv_path) # may need ("" , override=True)

user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
host = os.getenv("DB_HOST")
dbname = os.getenv("DB_NAME")

db = sqlalchemy.create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/{dbname}')

inspector = inspect(db)

# Get all schemas
schemas = inspector.get_schema_names()

for schema in schemas:
    print("Schema: ", schema)
    for table_name in inspector.get_table_names(schema=schema):
        print("Table: ", table_name)
    print("\n")

Schema:  information_schema
Table:  sql_packages
Table:  sql_features
Table:  sql_implementation_info
Table:  sql_parts
Table:  sql_languages
Table:  sql_sizing
Table:  sql_sizing_profiles


Schema:  public
Table:  django_migrations
Table:  django_content_type
Table:  auth_permission
Table:  auth_group
Table:  auth_group_permissions
Table:  account_user
Table:  account_user_groups
Table:  account_user_user_permissions
Table:  account_passwordresettoken
Table:  account_verifyemailtoken
Table:  django_admin_log
Table:  feedback_feedback
Table:  path_path
Table:  path_pathstep
Table:  path_pathdetail
Table:  path_userpathstate
Table:  path_pathstepdetail
Table:  path_university
Table:  path_userpathscore
Table:  path_userpathinteraction
Table:  path_pathcategory
Table:  path_pathstepcomplete
Table:  path_userpathfeedback
Table:  path_userpathstepfeedback
Table:  questionnaire_question
Table:  questionnaire_availableanswer
Table:  questionnaire_questioncategory
Table:  questionnaire_questi

In [26]:
# Establish a connection to the database
connection = db.connect()

# Function to check if user has SELECT permission on a table
def user_has_select_permission(user, schema, table):
    query = f"""
    SELECT has_table_privilege('{user}', '{schema}.{table}', 'SELECT');
    """
    result = connection.execute(query).scalar()
    return result

# Function to grant SELECT permission to user on a table
def grant_select_permission(user, schema, table):
    query = f"""
    GRANT SELECT ON {schema}.{table} TO {user};
    """
    connection.execute(query)

In [39]:
import ipywidgets as widgets
from IPython.display import display

# Get all tables in the database
all_tables = []
for schema in schemas:
    for table_name in inspector.get_table_names(schema=schema):
        all_tables.append(f"{schema}.{table_name}")

# Create a multi-select widget for tables
table_selector = widgets.SelectMultiple(
    options=all_tables,
    description='Tables:',
    disabled=False
)

selected_tables = []

def confirm_selection(button):
    global selected_tables
    selected_tables = list(table_selector.value)
    output_area.clear_output(wait=True)
    with output_area:
        print(f"Selected tables: {', '.join(selected_tables)}")

# Create an output area to display the selected tables
output_area = widgets.Output()

# Create a "Confirm Selection" button and assign the action
confirm_button = widgets.Button(description="Confirm Selection")
confirm_button.on_click(confirm_selection)

# Display the table selector, the confirm button, and the output area
display(table_selector, confirm_button, output_area)



SelectMultiple(description='Tables:', options=('information_schema.sql_packages', 'information_schema.sql_feat…

Button(description='Confirm Selection', style=ButtonStyle())

Output()

In [ ]:
from IPython.display import display, HTML, clear_output

# Define the action for the Grant button
def grant_permissions(button):
    clear_output(wait=True)
    messages = []
    for table in selected_tables:
        grant_select_permission(user, *table.split('.'))
        messages.append(f"Granted SELECT permission on {table}")
    
    display(HTML('<br>'.join(messages)))
    display(grant_button, revoke_button)

# Define the action for the Revoke button
def revoke_permissions(button):
    clear_output(wait=True)
    selected_tables = table_selector.value
    messages = []
    # for table in selected_tables:
        # revoke_select_permission(user, table)
        # messages.append(f"Revoked SELECT permission on {table}")
    
    # display(HTML('<br>'.join(messages)))
    display(grant_button, revoke_button)

# Create the Grant and Revoke buttons
grant_button = widgets.Button(description="Grant Permissions")
revoke_button = widgets.Button(description="Revoke Permissions")

# Assign the actions to the buttons
grant_button.on_click(grant_permissions)
revoke_button.on_click(revoke_permissions)

# Display the buttons
display(grant_button, revoke_button)

Button(description='Grant Permissions', style=ButtonStyle())

Button(description='Revoke Permissions', style=ButtonStyle())

In [38]:
# Test SELECT statement on all selected tables
if not selected_tables:
    print("No tables selected.")
else:
    print(f"Attempting to query the following tables: {', '.join(selected_tables)}")

    for table in selected_tables:
        try:
            query = f"SELECT * FROM {table} LIMIT 5"
            result = pd.read_sql(query, db)
            if result.empty:
                print(f"Table {table} is empty.")
            else:
                print(f"First 5 rows from {table}:")
                display(result)
        except Exception as e:
            print(f"Error querying table {table}: {e}")


No tables selected.


In [29]:
# Get table information
table_names = inspector.get_table_names(schema='public')
df_dict = {}
for table_name in table_names:
    try:
        parts = table_name.split('_')
        key = parts[0]
        if key not in df_dict:
            df_dict[key] = {}
        df_name = '_'.join(parts[1:])
        query = f'SELECT * FROM "{table_name}"'  # Use double quotes around table name
        df_dict[key][df_name] = pd.read_sql(query, con=db)
    except Exception as e:
        print(f"Error accessing table {table_name}: {str(e)}")

print(df_dict.keys())


Error accessing table django_migrations: (psycopg2.errors.InsufficientPrivilege) permission denied for table django_migrations

[SQL: SELECT * FROM "django_migrations"]
(Background on this error at: http://sqlalche.me/e/13/f405)
Error accessing table django_content_type: (psycopg2.errors.InsufficientPrivilege) permission denied for table django_content_type

[SQL: SELECT * FROM "django_content_type"]
(Background on this error at: http://sqlalche.me/e/13/f405)
Error accessing table auth_permission: (psycopg2.errors.InsufficientPrivilege) permission denied for table auth_permission

[SQL: SELECT * FROM "auth_permission"]
(Background on this error at: http://sqlalche.me/e/13/f405)
Error accessing table auth_group: (psycopg2.errors.InsufficientPrivilege) permission denied for table auth_group

[SQL: SELECT * FROM "auth_group"]
(Background on this error at: http://sqlalche.me/e/13/f405)
Error accessing table auth_group_permissions: (psycopg2.errors.InsufficientPrivilege) permission denied f

In [30]:
result = db.execute("SELECT current_user;")
for row in result:
    print("User:", row[0])

result = db.execute("SELECT table_name, privilege_type FROM information_schema.role_table_grants WHERE grantee=current_user;")
for row in result:
    print("Table:", row[0], "Privilege:", row[1])


Exception during reset or similar
Traceback (most recent call last):
  File "/Users/killianbrait/miniconda3/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 697, in _finalize_fairy
    fairy._reset(pool)
  File "/Users/killianbrait/miniconda3/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 893, in _reset
    pool._dialect.do_rollback(self)
  File "/Users/killianbrait/miniconda3/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 558, in do_rollback
    dbapi_connection.rollback()
psycopg2.OperationalError: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.



User: read_only


**Idea:** What if we could use the model to predict the top 5 plans for each newly registered user? What if we only show the user those 5 plans for their first 24 hours in the app. We ask them to do one of the plans and we show something happening. We connect the questionnaire directly to these five plans. I.e. what if when the user gets the questionnaire results applied to their plans, the plans shrink and fall away, leaving only a few that are then able to be completed and the user is able to log basic feedback on how the plan went. This can be used in a feedback loop to train the machine learning. We need to make this process fun and exciting.

Something like what this Juypter notebook will show.